In [2]:
import pandas as pd
df = pd.read_csv("Modified_SQL_Dataset.csv").sample(10000).reset_index(drop=True)
df

,Query,Label
0,"linares, 42",0
1,SELECT told FROM music,0
2,SELECT Count ( * ) AS future,0
3,"1%"" ) and sleep ( 5 ) and ( ""%"" = """,1
4,caesar-vitez2@apelton.sy,0
...,...,...
9995,"1%"" ) ) rlike ( select * from ( ...",1
9996,1' ) ) or char ( 68 ) ||char ( 6...,1
9997,fiset,0
9998,SELECT * FROM gone WHERE automobile = 'smaller...,0


In [2]:
import pandas as pd
import json
import urllib.parse
import base64

# 定义解码函数
def decode_sql(encoded_string):
    # 解码ASCII
    try:
        decoded_string = bytes.fromhex(encoded_string).decode('ascii')
    except:
        pass

    # 解码Unicode
    try:
        decoded_string = bytes.fromhex(encoded_string).decode('unicode_escape')
    except:
        pass

    # 解码JSON
    try:
        decoded_string = json.loads(encoded_string)
    except:
        pass

    # 解码URL
    try:
        decoded_string = urllib.parse.unquote(encoded_string)
    except:
        pass

    # 解码Base64
    try:
        decoded_string = base64.b64decode(encoded_string).decode('utf-8')
    except:
        pass

    return decoded_string

import pandas as pd
import re

# 定义大小写统一函数
def lowercase_sql(query):
    return query.lower()

# 定义泛化处理函数
def generalize_sql(query):
    # 将所有数字用"0"代替
    generalized_query = re.sub(r'\d+', '0', query)
    return generalized_query

# 定义分词函数
def tokenize_sql(query):
    # 在“<>”“!=”等特殊字符前加上空格
    query = re.sub(r'([<>!=])', r' \1 ', query)
    # 使用空格分隔词语
    tokens = query.split()
    return ' '.join(tokens)

# 对Query列进行处理，生成新列text
df['Text'] = df['Query'].apply(decode_sql)  # 解码处理
df['Text'] = df['Text'].apply(lowercase_sql)  # 大小写统一
df['Text'] = df['Text'].apply(generalize_sql)  # 泛化处理
df['Text'] = df['Text'].apply(tokenize_sql)  # 分词处理

print(df.head())


                                               Query  Label  \
0  SELECT * FROM Orders WHERE OrderDate BETWEEN '...      0   
1  SELECT TOP 3 * FROM name WHERE tiny = 'ice' SE...      0   
2  1"   )    )    rlike sleep  (  5  )   and    (...      1   
3                                   ; or '1'  =  '1'      1   
4  SELECT * FROM whom WHERE wait BETWEEN "adventu...      0   

                                                Text  
0  select * from orders where orderdate between '...  
1  select top 0 * from name where tiny = 'ice' se...  
2    0" ) ) rlike sleep ( 0 ) and ( ( "fzlr" = "fzlr  
3                                     ; or '0' = '0'  
4  select * from whom where wait between "adventu...  


In [3]:
## train test split 80%/20%
from sklearn.model_selection import train_test_split
train_df,test_df = train_test_split(df,test_size=0.20,random_state=50,shuffle=True)

train_df

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,Query,Label,Text
3357,SELECT AVG ( four ) FROM canal,0,select avg ( four ) from canal
9793,"1' ) ) union all select null,null,null...",1,"0' ) ) union all select null,null,null,null,nu..."
6002,"SELECT * FROM sides ORDER BY problem ASC, hun...",0,"select * from sides order by problem asc, hunt..."
5734,"-5243"" where 2105 = 2105 or 4144 = ( se...",1,"-0"" where 0 = 0 or 0 = ( select upper ( xmltyp..."
241,( select * from ( select ( sleep ( 5 ...,1,( select * from ( select ( sleep ( 0 ) ) ) srm...
...,...,...,...
8262,SELECT COUNT ( relationship ) FROM value,0,select count ( relationship ) from value
6214,1' in boolean mode ) and elt ( 1210 = 1...,1,"0' in boolean mode ) and elt ( 0 = 0,sleep ( 0..."
8324,2.23935E+15,0,0.0e+0
6253,1%' ) ) ) ( select ( case when...,1,0%' ) ) ) ( select ( case when ( 0 = 0 ) then ...


In [4]:
train_texts, train_labels = train_df['Text'].tolist(), train_df['Label'].tolist()
test_texts, test_labels = test_df['Text'].tolist(), test_df['Label'].tolist()

### Bert LSTM model

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel

# 定义自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(label, dtype=torch.long)
        } 

############### 定义BERT+LSTM模型
class BertLSTMClassifier(nn.Module):
    def __init__(self, bert_model, hidden_size, output_size, num_layers, bidirectional=True):
        super(BertLSTMClassifier, self).__init__()
        self.bert_model = bert_model
        self.lstm = nn.LSTM(bert_model.config.hidden_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, output_size)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)
        lstm_output, _ = self.lstm(outputs.last_hidden_state)
        lstm_output = self.dropout(lstm_output)
        logits = self.fc(lstm_output[:, -1, :])  # 取最后一个时刻的输出
        return logits


################### 定义BERT+TextCNN模型
class BertTextCNNClassifier(nn.Module):
    def __init__(self, bert_model, num_filters, filter_sizes, output_size):
        super(BertTextCNNClassifier, self).__init__()
        self.bert_model = bert_model
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=bert_model.config.hidden_size, out_channels=num_filters, kernel_size=fs) 
            for fs in filter_sizes
        ])
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(num_filters * len(filter_sizes), output_size)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.bert_model(input_ids=input_ids, attention_mask=attention_mask)

        # Transpose to fit the Conv1d input format (batch_size, num_channels, seq_len)
        embedded = outputs.last_hidden_state.transpose(1, 2)

        # Apply convolutional layers with different kernel sizes and max-pooling
        pooled_outputs = []
        for conv_layer in self.conv_layers:
            conv_out = nn.functional.relu(conv_layer(embedded))
            pooled_out, _ = torch.max(conv_out, dim=2)
            pooled_outputs.append(pooled_out)

        # Concatenate pooled outputs and flatten
        pooled_outputs = torch.cat(pooled_outputs, dim=1)
        pooled_outputs = self.dropout(pooled_outputs)

        # Fully connected layer for classification
        logits = self.fc(pooled_outputs)
        return logits


# 设置训练参数
batch_size = 64
max_length = 128
hidden_size = 128
num_layers = 1
# 根据实际情况设置类别数量
num_classes = 2
output_size = num_classes  
bidirectional = False

# 加载BERT模型和tokenizer
bert_model_name = 'bert-base-uncased'  # 使用中文预训练的BERT
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertModel.from_pretrained(bert_model_name)

# 创建数据集和数据加载器
train_dataset = CustomDataset(train_texts, train_labels, tokenizer, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset(test_texts, test_labels, tokenizer, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

############## 初始化LSTM模型
# model = BertLSTMClassifier(bert_model, hidden_size, output_size, num_layers, bidirectional=bidirectional)
################


################# 初始化TextCNN模型
num_filters = 100
filter_sizes = [2, 3, 4]
output_size = 2  # Number of classes in your classification task
model = BertTextCNNClassifier(bert_model, num_filters, filter_sizes, output_size)
########################


# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# 训练模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 5  # 根据实际情况设置训练轮数
from tqdm import tqdm  # 导入tqdm库，用于添加进度条

# 训练模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

print("Start training")
for epoch in range(num_epochs):
    total_loss = 0
    correct_train = 0
    total_train = 0

    # 使用tqdm包装train_loader，添加进度条
    with tqdm(train_loader, unit="batch") as t:
        for batch in t:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(logits.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            t.set_postfix({'loss': total_loss / (t.n + 1), 'accuracy': correct_train / total_train})


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# 测试模型
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []

    # 使用tqdm包装test_loader，添加进度条
    with tqdm(test_loader, unit="batch") as t:
        for batch in t:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            logits = model(input_ids, attention_mask)
            _, predicted = torch.max(logits.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

            t.set_postfix({})

# 计算评估指标
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Start training


100%|██████████| 32/32 [06:49<00:00, 12.80s/batch]

Test Accuracy: 0.9895
Precision: 0.9928
Recall: 0.9773
F1-score: 0.9850


## Word2Vec LSTM Pytorch

In [6]:
train_texts, train_labels = train_df['Text'].tolist(), train_df['Label'].tolist()
test_texts, test_labels = test_df['Text'].tolist(), test_df['Label'].tolist()

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from gensim.models import Word2Vec

# 定义自定义数据集类
class CustomDataset(Dataset):
    def __init__(self, texts, labels, word2vec_model, max_length):
        self.texts = texts
        self.labels = labels
        self.word2vec_model = word2vec_model
        self.word2vec_dim = word2vec_model.vector_size
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # 获取每个单词的Word2Vec嵌入向量
        words = text.split()
        embeddings = [self.get_word_embedding(word) for word in words]

        # 填充或截断Word2Vec嵌入向量，使其长度一致
        if len(embeddings) < self.max_length:
            embeddings.extend([torch.zeros(self.word2vec_dim, dtype=torch.float32)] * (self.max_length - len(embeddings)))
        else:
            embeddings = embeddings[:self.max_length]

        return {
            'word2vec_embeddings': torch.stack(embeddings),
            'labels': torch.tensor(label, dtype=torch.long)
        }

    def get_word_embedding(self, word):
        if word in self.word2vec_model.wv:
            return torch.tensor(self.word2vec_model.wv[word], dtype=torch.float32)
        else:
            return torch.zeros(self.word2vec_dim, dtype=torch.float32)

############### 定义Word2Vec+LSTM模型
class Word2VecLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, bidirectional=True):
        super(Word2VecLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, bidirectional=bidirectional, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(hidden_size * 2 if bidirectional else hidden_size, output_size)

    def forward(self, word2vec_embeddings):
        lstm_output, _ = self.lstm(word2vec_embeddings)
        lstm_output = self.dropout(lstm_output)
        logits = self.fc(lstm_output[:, -1, :])  # 取最后一个时刻的输出
        return logits
    
################### 定义Word2Vec+TextCNN模型
class Word2VecTextCNNClassifier(nn.Module):
    def __init__(self, input_size, num_filters, filter_sizes, output_size):
        super(Word2VecTextCNNClassifier, self).__init__()
        self.num_filters = num_filters
        self.filter_sizes = filter_sizes
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=input_size, out_channels=num_filters, kernel_size=fs) 
            for fs in filter_sizes
        ])
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(num_filters * len(filter_sizes), output_size)

    def forward(self, word2vec_embeddings):
        # Transpose to fit the Conv1d input format (batch_size, num_channels, seq_len)
        embedded = word2vec_embeddings.transpose(1, 2)

        # Apply convolutional layers with different kernel sizes and max-pooling
        pooled_outputs = []
        for conv_layer in self.conv_layers:
            conv_out = nn.functional.relu(conv_layer(embedded))
            pooled_out, _ = torch.max(conv_out, dim=2)
            pooled_outputs.append(pooled_out)

        # Concatenate pooled outputs and flatten
        pooled_outputs = torch.cat(pooled_outputs, dim=1)
        pooled_outputs = self.dropout(pooled_outputs)

        # Fully connected layer for classification
        logits = self.fc(pooled_outputs)
        return logits

    

# 训练Word2Vec模型
sentences = [text.split() for text in train_texts + test_texts]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

batch_size = 64
max_length = 10

# 创建数据集和数据加载器
train_dataset = CustomDataset(train_texts, train_labels, word2vec_model, max_length)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset(test_texts, test_labels, word2vec_model, max_length)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

########### 初始化Word2vec LSTM模型
# 设置训练参数
input_size = word2vec_model.vector_size
hidden_size = 128
num_layers = 1
output_size = 2  # 根据实际情况设置类别数量
bidirectional = True
model = Word2VecLSTMClassifier(input_size, hidden_size, output_size, num_layers, bidirectional=bidirectional)
#############

input_size = word2vec_model.vector_size  # Assuming each Word2Vec embedding has a size of 300
num_filters = 100
filter_sizes = [2, 3, 4]
output_size = 2  # Number of classes in your classification task
# ######### 初始化Word2VecTextCNNClassifier
model = Word2VecTextCNNClassifier(input_size, num_filters, filter_sizes, output_size)
####################

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

# 训练模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

num_epochs = 5  # 根据实际情况设置训练轮数
from tqdm import tqdm  # 导入tqdm

# 训练模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

print("Start training")
for epoch in range(num_epochs):
    total_loss = 0
    correct_train = 0
    total_train = 0

    # 使用tqdm包装train_loader，添加进度条
    with tqdm(train_loader, unit="batch") as t:
        for batch in t:
            word2vec_embeddings = batch['word2vec_embeddings'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            logits = model(word2vec_embeddings)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(logits.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            t.set_postfix({'loss': total_loss / (t.n + 1), 'accuracy': correct_train / total_train})

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# 测试模型
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []

    # 使用tqdm包装test_loader，添加进度条
    with tqdm(test_loader, unit="batch") as t:
        for batch in t:
            word2vec_embeddings = batch['word2vec_embeddings'].to(device)
            labels = batch['labels'].to(device)

            logits = model(word2vec_embeddings)
            _, predicted = torch.max(logits.data, 1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

            t.set_postfix({})

# 计算评估指标
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Start training


100%|██████████| 32/32 [00:00<00:00, 66.23batch/s]


Test Accuracy: 0.9680
Precision: 0.9805
Recall: 0.9276
F1-score: 0.9533
